This lab on Cross-Validation is a python adaptation of p. 190-194 of "Introduction to Statistical Learning with Applications in R" by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani. Written by R. Jordan Crouser at Smith College for SDS293: Machine Learning (Spring 2016) and is inspired from http://www.science.smith.edu/~jcrouser/SDS293/labs/lab9-py.html

In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
import itertools
import statsmodels.api as sm
import matplotlib.pyplot as plt

## data exploration

In [4]:
df = pd.read_csv('Hitters.csv')
df

FileNotFoundError: ignored

In [ ]:
df.info()

In [ ]:
df.head()

## Data preperation

In [ ]:
# Drop any rows the contain missing values, along with the player names
df = df.dropna()

# Get dummy variables # This a conversion for the categorical to be in 
dummies = pd.get_dummies(df[['League', 'Division', 'NewLeague']])
print(dummies)


# Extract The target variable # 
y = pd.DataFrame(df.Salary)

# Drop the column with the independent variable (Salary), and columns for which we created dummy variables
X_ = df.drop(['Salary', 'League', 'Division', 'NewLeague'], axis=1).astype('float64')

# Define the feature set X.
X = pd.concat([X_, dummies[['League_N', 'Division_W', 'NewLeague_N']]], axis=1)

In [ ]:
np.random.seed(seed=12)
print(len(y))
train = np.random.choice([True, False], size = len(y), replace = True)
test = np.invert(train)
#print(train[1:10],test[1:10])

## Modeling and feature selection

In [ ]:
def processSubset(feature_set, X_train, y_train, X_test, y_test):
    # Fit model on feature_set and calculate RSS
    model = sm.OLS(y_train,X_train[list(feature_set)]) # Linear regression with ordinary least square 
    regr = model.fit()
    RSS = ((regr.predict(X_test[list(feature_set)]) - y_test) ** 2).sum()
    return {"model":regr, "RSS":RSS}

In [ ]:
def forward(predictors, X_train, y_train, X_test, y_test):
    
    results = []

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X_train.columns if p not in predictors]
    
    # 
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p], X_train, y_train, X_test, y_test))

    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    #display(models)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    #print(best_model)
    
    # Return the best model, along with some other useful information about the model
    return best_model

In [ ]:
models_train = pd.DataFrame(columns=["RSS", "model"])

predictors = []

for i in range(1,len(X.columns)+1):    
    models_train.loc[i] = forward(predictors, X[train], y[train]["Salary"], X[test], y[test]["Salary"])
    #print(model_train)
    predictors = models_train.loc[i]["model"].model.exog_names
    #print(predictors)


In [ ]:
plt.plot(models_train["RSS"])
plt.xlabel('number of Predictors')
plt.ylabel('RSS')
#plt.plot(models_train["RSS"].argmin, models_train["RSS"].min(), "or")

## Forward subset selection with cross validation

In [ ]:
k = 10        # number of folds
np.random.seed(seed=1)
folds = np.random.choice(k, size = len(y), replace = True)

# Create a DataFrame to store the results of our upcoming calculations
cv_errors = pd.DataFrame(columns=range(1,k+1), index=range(1,20))
cv_errors = cv_errors.fillna(0)

In [ ]:
import collections
counter=collections.Counter(folds)
print(counter)
print(folds)

In [ ]:
models_cv = pd.DataFrame(columns=["RSS", "model"])
    
# Outer loop iterates over all folds
for j in range(1,k+1):

    # Reset predictors
    predictors = []
    
    # Inner loop iterates over each size i
    for i in range(1,len(X.columns)+1):    
    
        # The perform forward selection on the full dataset minus the jth fold, test on jth fold
        models_cv.loc[i] = forward(predictors, X[folds != (j-1)], y[folds != (j-1)]["Salary"], X[folds == (j-1)], y[folds == (j-1)]["Salary"])
        
        # Save the cross-validated error for this fold
        cv_errors[j][i] = models_cv.loc[i]["RSS"]

        # Extract the predictors
        predictors = models_cv.loc[i]["model"].model.exog_names
        

In [ ]:
predictors[0:10]

In [ ]:
cv_mean = cv_errors.apply(np.mean, axis=1)

plt.plot(cv_mean)
plt.xlabel('# Predictors')
plt.ylabel('CV Error')
plt.plot(cv_mean.argmin(), cv_mean.min(), "or")

In [ ]:
cv_mean

In [ ]:
print(models_cv.loc[9, "model"].summary())

## Using the sklearn feature selection functions

Notes: 
1. The built in sklearn.feature_selection.SequentialFeatureSelector uses cross validation within its algorithm, in addition it needs to specify the number output features.
2. mlxtend library is another option to do feature selection.


### Sequential feature selection (forward / backword)

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
import sklearn.linear_model as skl_lm
# Linear
lm = skl_lm.LinearRegression()


In [ ]:
sfs = SequentialFeatureSelector(lm, n_features_to_select=9 ,direction = 'forward' ,cv =10 , scoring = 'neg_mean_squared_error')

In [ ]:
sfs.fit(X[train], y[train])

In [ ]:
sfs.get_support()

In [ ]:
X.columns

### Recursive Feature selection

In [ ]:
from sklearn.feature_selection import RFECV

In [ ]:
selector = RFECV(lm, step=1 , cv  = 10) # step indicate the number of feature to elemintate each time.

In [ ]:
selector = selector.fit(X[train], y[train])

In [ ]:
selector.support_